In [ ]:
"""Please keep in mind that this whole document is only
to give a general idea of what to implement. 
These are not functioning codes."""

In [1]:
import nltk

In [2]:
import spacy

In [3]:
from gensim.parsing.preprocessing import remove_stopwords

In [31]:
from elasticsearch import Elasticsearch

In [ ]:
# test code for formality. Does not work on my laptop.
es_host = "http://localhost:9200"
es_user = "elastic"
es_password = "1234"

es = Elasticsearch(
    hosts=es_host,  # "http://localhost:9200"
    http_auth=(es_user, es_password)   # credentials for basic authentication
)

In [4]:
# draft : implementation step
######################################
# translate regular sentence to slang#
######################################

input_str = "I have something to say"

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
str_doc = nlp(input_str)

In [9]:
len(str_doc)

5

In [30]:
for np in str_doc.noun_chunks:
    print(np)
# print noun phrases.

I
something


In [32]:
"""To find and return subject of a preprocessed sentence."""
# I wrote the function based on which one may
# omit the subject of a sentence with a possibility. 
# This is not used now, as I still don't
# know how to calculate the weight of such omission.
def subject_find(doc):
    for token in doc:
        if "subj" in token.dep_:
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [23]:
# do slang dictionary query with part of the input sentence
"""returns all possible k-word-shingle for the input sentence.
form: list, [[1-shingle][2-shingle]...[k-shingle]]""" 
def multiple_shingle(doc_str):
    doc = doc_str.split(" ")
    shingle_lst=[]
    for k in range(0,len(doc)):
        shingle_lst_temp = []
        for i in range(len(doc)-k):
            shingle_lst_temp.append(doc[i:k+i+1])
        shingle_lst.append(shingle_lst_temp)
    return shingle_lst

In [24]:
multiple_shingle(input_str)

[[['I'], ['have'], ['something'], ['to'], ['say']],
 [['I', 'have'], ['have', 'something'], ['something', 'to'], ['to', 'say']],
 [['I', 'have', 'something'],
  ['have', 'something', 'to'],
  ['something', 'to', 'say']],
 [['I', 'have', 'something', 'to'], ['have', 'something', 'to', 'say']],
 [['I', 'have', 'something', 'to', 'say']]]

In [ ]:
"""search in slang database based on shingles,
return slang and the corresponding shingle.
NOTE: not finished. """
def slang_search(shingle_lst):
    lst_search_result = []
    for i in range(len(shingle_lst)):
        list_temp = shingle_lst[i]
        for j in range(len(list_temp)):
            search_result = es.search(index="slang",query={"match": {
                              "expansion": list_temp[j]}})
            # TODO: make it a fuzzy search
            if search_result["hits"]["hits"]:
                for hit in search_result["hits"]["hits"]:
                    lst_search_result.append((list_temp[j],hit["abbreviation"]))
    return lst_search_result

In [34]:
"""remove stop words,
perform a word association based on twitter corpus.
perform ES query based on associated words.
return: corresponding word and associated emoji as list of tuple.
NOTE: not finished. """

def emoji_search(input_str):
    emoji_search_results = []
    en_stops = set(nltk.corpus.stopwords.words('english'))
    word_list = [x.strip() for x in input_str.split(" ")]
    model = Word2Vec.load('model.bin')
    for word in word_list:
        if word not in en_stops:
            # perform association
            association_list = model.wv.most_similar(word)
            # perform emoji search
            for word_associated in association_list:
            # TODO: search for emoji with keywords related
            # store in list_of_emoji
            # TODO: return word_associated with associated emoji
                emoji_search_results.append(word,list_of_emoji)
    return emoji_search_results
    # TODO: can also based on api to reversedictionary.com
    # which gives associated words to noun phrases: 
    # 1. extract noun phrases with nlp().noun_chunks,2.search
    # the phrase up in reversedictionary.com, and
    # 3.perform query with ES in emoji dataset with obtained 
    # set of words.

In [ ]:
"""The general function that would 
translate input to sentence with slang&emoji.
return&print translated sentence.
To be continued.
"""
def translation_style(input_str):
    